In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

from operator import itemgetter
from dotenv import load_dotenv
import os


In [2]:

# Charger les variables depuis le fichier .env
load_dotenv()

# Définir les variables d'environnement
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Tu peux aussi ajouter cette variable dans ton fichier .env si tu veux
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus/"

# Récupérer la clé API depuis .env
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Optionnel: Vérifier si la clé API est bien chargée
if os.environ["LANGCHAIN_API_KEY"] is None:
    raise ValueError("LANGCHAIN_API_KEY is not set. Please check your .env file.")


In [3]:

#call the model with ChatOllama
# local_model = "qwen2:1.5b"
local_model = "phi3"

llm = ChatOllama(model=local_model)

#Parse the response
parser = StrOutputParser()

 # Instantiate embedding model
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)

# VectorstoreChroma
persist_directory = './db_qsar_bdii'

vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
print(vectorstore._collection.count())

0


In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [ ]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)




In [5]:
question = "who is el mansour edahbi?"
result = qa.invoke({"input": question})
result["answer"]

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it]


"Elahi Mansour Edahbib, also known as El Mansour Edahbi or just E.M., was a prominent figure in Iranian politics before he went missing under mysterious circumstances on May 8, 2019. Born on October 3, 1967, in Tehran, Iran, Edathiab served variously as the Minister for Education and Culture (from August to September 2013) during Hassan Rouhani's presidency, before being replaced by Mohammad Hossein Khatibi. His sudden disappearance created a wave of concern across political communities in Iran, leading many media outlets around the world to speculate about his fate and raise questions regarding press freedom within the country. However, as of my knowledge cutoff date in early 2023, E.M.'s whereabouts remain unknown with no official updates on his situation from Iranian authorities or international human rights organizations monitoring developments."

In [6]:
question = "in brief"
result = qa.invoke({"input": question})
result["answer"]

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


'Alice encounters a Cheshire Cat in Wonderland. The cat provides her with cryptic guidance about seeking answers from others but emphasizes that she will find them elsewhere, suggesting an allegorical message regarding self-reliance and the search for truth beyond social interactions. This scenario mirrors Rousseau\'s belief that humans can inherently understand goodness without external influence while critiquing overdependence on society as portrayed by Victorian England through Carroll’s satire in "Alice" storyline, resonating with existential themes of self-discovery and authenticity.'